# Structure Function Turbulent Data

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from turboflow.models.rff import RFFNet
from turboflow.dataloaders import Turbo2D_simple, load_turbo2D_simple_numpy
from torch.utils.data import Dataset, DataLoader

# set up cuda 
torch.cuda.is_available()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [8]:
Xlr, Ulr = load_turbo2D_simple_numpy(ds=4, img=42) # low resolution (64x64)
L = int(Xlr.shape[0]**0.5)
print(L)

delta_x = Xlr[1,1] - Xlr[0,1]
print(delta_x)

Y shape (64, 64, 2)
Y min, max: -2.365612 2.941536
after normalization, Y min, max: -0.8042097733972999 1.0
64
0.01568627450980392


In [33]:
patch_dim = 32
min_l = delta_x/4

patch_ln = torch.arange(-min_l*patch_dim, min_l*patch_dim, min_l, device=device)

print(patch_ln)
print(patch_ln.shape)
print(patch_ln[1] - patch_ln[0])
print(patch_ln[4] - patch_ln[0])

patch_sq = torch.stack(torch.meshgrid(patch_ln, patch_ln), dim=-1)
print(patch_sq.shape)

tensor([-0.1255, -0.1216, -0.1176, -0.1137, -0.1098, -0.1059, -0.1020, -0.0980,
        -0.0941, -0.0902, -0.0863, -0.0824, -0.0784, -0.0745, -0.0706, -0.0667,
        -0.0627, -0.0588, -0.0549, -0.0510, -0.0471, -0.0431, -0.0392, -0.0353,
        -0.0314, -0.0275, -0.0235, -0.0196, -0.0157, -0.0118, -0.0078, -0.0039,
         0.0000,  0.0039,  0.0078,  0.0118,  0.0157,  0.0196,  0.0235,  0.0275,
         0.0314,  0.0353,  0.0392,  0.0431,  0.0471,  0.0510,  0.0549,  0.0588,
         0.0627,  0.0667,  0.0706,  0.0745,  0.0784,  0.0824,  0.0863,  0.0902,
         0.0941,  0.0980,  0.1020,  0.1059,  0.1098,  0.1137,  0.1176,  0.1216],
       device='cuda:0')
torch.Size([64])
tensor(0.0039, device='cuda:0')
tensor(0.0157, device='cuda:0')
torch.Size([64, 64, 2])


In [12]:
from turboflow.models.phyrff_sfun import SfunRFFNet

# model (RFF + MLP)
fft_scale = 10
fft_nfeat = 256
nin = 2  # = x and y coordinates
nout = 2 # = ux and uy components
mlp_layers = [nin*fft_nfeat] + [256]*3 + [nout]

smallest_increment = delta_x/4

rffnet = SfunRFFNet('RFFNet', mlp_layers, fft_nfeat, fft_scale,
                    smallest_increment = smallest_increment)

1/0
# training data
path_to_data = '../data/2021-Turb2D_velocities.npy'
trainset = Turbo2D_simple(path_to_data, device, ds=4, img=42)
trainloader = DataLoader(trainset, batch_size=6666, shuffle=False)

[0.00392156862745098, 0.00784313725490196, 0.011764705882352941, 0.01568627450980392]


ZeroDivisionError: division by zero

In [ ]:
# TRAIN!
rffnet.to(device)
rffnet.fit(trainloader, epochs=2000) # we are not afraid to overfit the data, this is coord-based MLP!